In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
from sklearn import preprocessing
import numpy as np
from statsmodels.stats import inter_rater as irr
pd.options.display.max_colwidth = 240

In [38]:
juanjo = pd.read_csv('../Data/Data_Annotated/recent_juanjo_annotations.csv')
juanma = pd.read_csv('../Data/Data_Annotated/recent_juanma_annotations.csv')

In [45]:
juanjo = juanjo.rename(columns = {'sentimento':'sentiment_juanjo', 'relevancia': 'relevancia_juanjo', 'emocion': 'emocion_juanjo', 'ironia': 'ironia_juanjo'})
juanma = juanma.rename(columns = {'sentimento':'sentiment_juanma', 'relevancia': 'relevancia_juanma', 'emocion': 'emocion_juanma', 'ironia': 'ironia_juanma'})

In [46]:
merged_df = pd.merge(juanjo[['text', 'tweet_id', 'sentiment_juanjo']], juanma[['tweet_id', 'sentiment_juanma']], on=['tweet_id'])
merged_relevancia = pd.merge(juanjo[['text', 'tweet_id', 'relevancia_juanjo']], juanma[['tweet_id', 'relevancia_juanma']], on=['tweet_id'])
merged_emocion = pd.merge(juanjo[['text', 'tweet_id', 'emocion_juanjo']], juanma[['tweet_id', 'emocion_juanma']], on=['tweet_id'])
merged_ironia = pd.merge(juanjo[['text', 'tweet_id', 'ironia_juanjo']], juanma[['tweet_id', 'ironia_juanma']], on=['tweet_id'])

In [6]:
dict_emociones = {
    'alegria' : ['Alegria', 'Confianza', 'Admiración','Agrado'],
    'miedo' : ['Miedo', 'Incertidumbre'],
    'sorpresa' : ['Sorpresa', 'Asombro'],
    'tristeza' : ['Tristeza' , 'Decepción'],
    'asco' : ['Asco', 'Desagrado'],
    'ira' : ['Ira', 'Odio'],
    'otra' : ['Otra'] }

In [7]:
for i  in dict_emociones.keys():
    merged_df[i + '_juanma'] = merged_df['sentiment_juanma'].str.contains('|'.join(dict_emociones[i]))
    merged_df[i + '_juanjo'] = merged_df['sentiment_juanjo'].str.contains('|'.join(dict_emociones[i]))

In [8]:
merged_df = merged_df.fillna(False)

In [9]:
merged_df['labels'] = merged_df.apply(lambda x: ', '.join(x.index[x == True]), axis=1)

In [11]:
raw_counts = {}
for i in dict_emociones.keys():
    for j in dict_emociones[i]:
        raw_emotion = j
        raw_counts[j] = len(merged_df[(merged_df['sentiment_juanjo'].str.contains(raw_emotion).fillna(False)) | (merged_df['sentiment_juanma'].str.contains(raw_emotion).fillna(False))]) 

In [23]:
raw_emotion = 'Agrado'
merged_df[(merged_df['sentiment_juanjo'].str.contains(raw_emotion).fillna(False)) | (merged_df['sentiment_juanma'].str.contains(raw_emotion).fillna(False))]

,text,tweet_id,sentiment_juanjo,sentiment_juanma,Confianza_juanma,Confianza_juanjo,miedo_juanma,miedo_juanjo,sorpresa_juanma,sorpresa_juanjo,...,asco_juanjo,ira_juanma,ira_juanjo,expectativa_juanma,expectativa_juanjo,alegria_juanma,alegria_juanjo,otra_juanma,otra_juanjo,labels
0,"#NosUnimosONosJodemos es una persona de fiar, no tiene rabo de paja, es un buen administrador y gestor, une, da confianza y hace la politica que es a favor del pueblo, es temeroso de Dios.",1528357012517621762,"{""choices"": [""Admiración"", ""Agrado""]}","{""choices"": [""Admiración"", ""Agrado"", ""Confianza""]}",True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"
2,"@SaraLop88370165 @FicoGutierrez La propuesta de Fico está basada y planificada, el tiene la capacidad de asumir la presidencia #FicoGanador #ElMejorCandidatoEs",1529826360180346880,Agrado,"{""choices"": [""Admiración"", ""Confianza""]}",True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"
5,"@enriqueyimis1 @WRadioColombia #ElMejorCandidatoEs de lejos la dupla Petro y Francia. Porque se convirtieron en el referente del país que nos incluye a todos. No solo los que somos abiertamente antifascistas, sino a los ciudadanos comun...",1529785193804050433,Agrado,"{""choices"": [""Admiración"", ""Confianza""]}",True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"
7,"#MiVotoEsPorFICO #SantandereanosConFICO porq conectará el campo triplicando la inversión en vías terciarias para que nuestros campesinos puedan sacar sus productos a los mercados #FicoElGanador Nuevo simulador de votación,eligiendo a @...",1529933656093081636,Agrado,Confianza,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"
20,#VotoPor Sergio Fajardo. Un presidente decente y que no divida es lo que necesita Colombia,1529613037866459136,Agrado,Admiración,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"
63,#ElDebateDefinitivo viendo a Fico sin una palabra significativa. #YoVotoPorFajardo por sus palabras sensatas y posturas claras. El único sin lugares comunes!,1528903235209150465,Agrado,"{""choices"": [""Admiración"", ""Sorpresa""]}",True,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo, sorpresa_juanma"
80,"Isssss @petrogustavo llamará a la ONU para que investigue la corrupción, se va a caer toda la estantería de la corrupción en Colombia. #MiVotoEsPetroYFrancia",1528543164067459072,"{""choices"": [""Expectativa"", ""Agrado""]}","{""choices"": [""Expectativa"", ""Confianza""]}",True,True,False,False,False,False,...,False,False,False,True,True,False,False,False,False,"Confianza_juanma, Confianza_juanjo, expectativa_juanma, expectativa_juanjo"
90,Patria Honor lealtad !! #FedericoEsColombia,1528503220875714560,Agrado,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Confianza_juanjo


In [12]:
raw_counts

{'Confianza': 34,
 'Aceptación': 1,
 'Admiración': 16,
 'Agrado': 8,
 'Miedo': 8,
 'Sorpresa': 3,
 'Incertidumbre': 6,
 'Asombro': 2,
 'Tristeza': 1,
 'Asco': 22,
 'Desagrado': 30,
 'Odio': 6,
 'Decepción': 17,
 'Indiferencia': 0,
 'Ira': 8,
 'Expectativa': 15,
 'Vigilancia': 0,
 'Alegria': 5,
 'Calma': 1,
 'Otra': 6}

In [130]:
raw_emotion = 'Aceptación'
merged_df[(merged_df['sentiment_juanjo'].str.contains(raw_emotion)) | (merged_df['sentiment_juanma'].str.contains(raw_emotion))]

,text,tweet_id,sentiment_juanjo,sentiment_juanma,Confianza_juanma,Confianza_juanjo,miedo_juanma,miedo_juanjo,sorpresa_juanma,sorpresa_juanjo,...,asco_juanjo,ira_juanma,ira_juanjo,expectativa_juanma,expectativa_juanjo,alegria_juanma,alegria_juanjo,otra_juanma,otra_juanjo,labels
62,"Fajardo comienza el debate mostrando como las agresiones y señalamientos entre Fico y Petro generan una afectación a la población colombiana, afectación en la cual él no quiere participar #YoVotoPorFajardo #ElDebateDefinitivo #DebateEl...",1528905779373391872,Aceptación,Admiración,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,"Confianza_juanma, Confianza_juanjo"


In [24]:
algo = 'sorpresa'
merged_df[(merged_df[algo + '_juanma'] == True) & (merged_df[algo + '_juanjo'] == True)]

,text,tweet_id,sentiment_juanjo,sentiment_juanma,alegria_juanma,alegria_juanjo,miedo_juanma,miedo_juanjo,sorpresa_juanma,sorpresa_juanjo,tristeza_juanma,tristeza_juanjo,asco_juanma,asco_juanjo,ira_juanma,ira_juanjo,otra_juanma,otra_juanjo,labels


In [14]:
df_list = []
emotions_count = {}
for i in dict_emociones.keys():
    emotion_df = merged_df[(merged_df[i + '_juanma']== True) | 
                           (merged_df[i + '_juanjo']== True)][['text', 'labels', 'tweet_id' , i + '_juanma', i +'_juanjo', 'sentiment_juanma', 'sentiment_juanjo']]
    emotion_df = emotion_df.rename(columns= {i + '_juanma': 'juanma', i +'_juanjo': 'juanjo'})
    emotion_df['emocion'] = i
    emotions_count[i] = len(emotion_df)
    df_list.append(emotion_df)
full_df = pd.concat(df_list, ignore_index=True)
full_df_disagreement = full_df[full_df['juanma'] != full_df['juanjo']]
full_df_agreement = full_df[full_df['juanma'] == full_df['juanjo']]

In [35]:
full_df_agreement[full_df_agreement['emocion'] == 'alegria']

,text,labels,tweet_id,juanma,juanjo,sentiment_juanma,sentiment_juanjo,emocion
0,"#VotoPor #RodolfoHernandez hombre sabio, sagaz y visionario que transformará el país en potencia mundial, del atraso a ejemplo de desarrollo y crecimiento económico #RodolfoPresidente A los haters les digo, #RodolfoPresidente y si van a decir algo con argumentos no inventos","alegria_juanma, alegria_juanjo",1529572795633414144,True,True,Admiración,Confianza,alegria
1,#FedericoEsColombia este 29 de Mayo salgamos todos a votar por la democracia y la libertad de colombia unidos por nuestro país #FedericoImparable #FedericoEsColombia #FicoPresidente2022,"alegria_juanma, alegria_juanjo",1528415123450343432,True,True,Confianza,Confianza,alegria
2,"Por un cambio, por la esperanza de un mejor país, por la vida de líderes jóvenes y líderes sociales, #MiVotoEsPetroYFrancia","alegria_juanma, alegria_juanjo",1528473443271835653,True,True,Confianza,Confianza,alegria
3,Por que Colombia necesita cambio #MiVotoEsPetroYFrancia,"alegria_juanma, alegria_juanjo",1528584113933934593,True,True,Confianza,Confianza,alegria
4,"#ElMejorCandidatoEs el que le apuesta por la paz, la justicia social y un gobierno para todos.","alegria_juanma, alegria_juanjo",1529768426037821443,True,True,Confianza,Confianza,alegria
7,Es claro quien es el presidente y quienes van a quedar rabones ! Petro los peinó tranquilo ! #ElDebateDefinitivo,"alegria_juanma, alegria_juanjo",1529144849441644545,True,True,Confianza,Agrado,alegria
10,"Ahora que #PetroEsPresidente #NuestraPrioridadEs hacerle seguimiento. Apoyarlo con lo bueno, reprocharle lo malo. Marchar cuando sea necesario y así... Será difícil, pero no imposible. Pero con responsabilidad, no con odio.","alegria_juanma, alegria_juanjo",1539264520631242753,True,True,Confianza,Confianza,alegria
11,¿Cuándo un presidente nos había dicho que nos quiere mucho? Y es cierto 🥺. #PetroEsPresidente.,"alegria_juanma, alegria_juanjo",1538916021280681984,True,True,"{""choices"": [""Alegria"", ""Confianza""]}",Alegria,alegria
13,"¡LLEGO LA HORA DE VIVIR SABROSO, CARAJO! #PetroPresidente #VivirSabroso ❤️❤️❤️","alegria_juanma, alegria_juanjo",1538655871353372673,True,True,"{""choices"": [""Alegria"", ""Confianza""]}",Alegria,alegria
14,"La verdad no pensé que viviría para ver el inicio del cambio...gracias por todo colombianos conscientes y pensantes, los amo 😭😭😭 #PetroEsPresidente","alegria_juanma, alegria_juanjo",1538667002583306245,True,True,"{""choices"": [""Alegria"", ""Confianza""]}",Alegria,alegria


In [65]:
pd.options.display.max_colwidth = 300
full_df_disagreement[full_df_disagreement['emocion'] == 'sorpresa']

,text,labels,tweet_id,juanma,juanjo,sentiment_juanma,sentiment_juanjo,emocion
66,@yescuder #PetroEsPresidente | Le recuerdo que para esta segunda vuelta salió a votar las tribus e indígenas más excluidos en nuestro país. Indígenas total en nuestro país registrado: 1'905.617 de personas También le recuerdo Sigue el Hilo 🧵,sorpresa_juanjo,1538918509450764290,False,True,False,Sorpresa,sorpresa
67,Veo con emoción que muchos de mis amigos reportan masiva asistencia a las urnas. Colombia sí está cambiando. El pueblo despertó. ✊🏻 #Elecciones2022 #EleccionesColombia,"alegria_juanma, alegria_juanjo, sorpresa_juanjo",1530971942110609408,False,True,Confianza,"{""choices"": [""Asombro"", ""Alegria""]}",sorpresa
68,La verdad yo si quisiera saber que le hizo @AlvaroUribeVel a @petrogustavo que no lo suelta para nada. Todo el #ElDebateDefinitivo se la paso uribe uribe uribe y nada mas. Ojalá me respondiera,"sorpresa_juanjo, tristeza_juanjo, asco_juanma",1528956465632903168,False,True,Desagrado,"{""choices"": [""Decepción"", ""Sorpresa""]}",sorpresa


In [138]:
full_df_agreement[full_df_agreement['emocion'] == 'alegria']

,text,labels,juanma,juanjo,sentiment_juanma,sentiment_juanjo,emocion


In [ ]:
full_df[full_df['emocion'].str.contains('')]

In [10]:
merged_df.to_csv('../Data/Data_Annotated/merged_annotations.csv', index=False)

In [41]:
merged_relevancia = merged_relevancia.fillna('No')
merged_relevancia['relevancia_juanjo_bool'] = merged_relevancia['relevancia_juanjo'].map({'Si': True, 'No': False})
merged_relevancia['relevancia_juanma_bool'] = merged_relevancia['relevancia_juanma'].map({'Si': True, 'No': False})

In [42]:
arr = np.array((merged_relevancia['relevancia_juanjo_bool'], merged_relevancia['relevancia_juanma_bool'])).T
agg = irr.aggregate_raters(arr)
irr.fleiss_kappa(agg[0], method='fleiss')

1.0

In [54]:
merged_relevancia[merged_relevancia['relevancia_juanjo'] != merged_relevancia['relevancia_juanma']]

,text,tweet_id,relevancia_juanjo,relevancia_juanma


In [57]:
merged_emocion = merged_emocion.fillna('No')
merged_emocion['emocion_juanjo_bool'] = merged_emocion['emocion_juanjo'].map({'Si': True, 'No': False})
merged_emocion['emocion_juanma_bool'] = merged_emocion['emocion_juanma'].map({'Si': True, 'No': False})

In [59]:
arr_num = np.array((merged_emocion['emocion_juanjo_bool'], merged_emocion['emocion_juanma_bool'])).T
agg_num = irr.aggregate_raters(arr_num)
irr.fleiss_kappa(agg_num[0], method='fleiss')

0.37751388622869153

In [58]:
merged_emocion[merged_emocion['emocion_juanjo'] != merged_emocion['emocion_juanma']]

,text,tweet_id,emocion_juanjo,emocion_juanma,emocion_juanjo_bool,emocion_juanma_bool
1,#FedericoEsColombia este 29 de Mayo salgamos todos a votar por la democracia y la libertad de colombia unidos por nuestro país #FedericoImparable #FedericoEsColombia #FicoPresidente2022,1528415123450343432,No,Si,False,True
4,"#LeApuestoA las instituciones. Nunca he creído en el mesianismo. #LeApuestoA al debate y pensamiento crítico para construir en la diferencia, sin mojigatería, ni límites absurdos sobre el deber ser.",1529097150923214849,Si,No,True,False
27,Ya convencí a 10 personas #ConvenceUnoMásPorPetro,1529573334907768832,No,Si,False,True
28,#FicoElGanador y con el gana Colombia.,1529850058417618946,No,Si,False,True
32,@yescuder #PetroEsPresidente | Le recuerdo que para esta segunda vuelta salió a votar las tribus e indígenas más excluidos en nuestro país. Indígenas total en nuestro país registrado: 1'905.617 de personas También le recuerdo Sigue el Hilo 🧵,1538918509450764290,Si,No,True,False
44,"Entre el #ExGuerrilleroOIngeniero yo, Natalia, me quedo con el ingeniero ¿y usted? (Copia y pega con tu nombre)",1538235365953265666,No,Si,False,True
52,"#NosHaceFalta que @JuanPabloCalvas en su programa ""Sigue la W"", haga un programa sobre los lideres sociales asesinados en los últimos 4 años. Hace dos días se emitió un programa analizando un caso, pero faltan muchos mas.",1537413012411273218,Si,No,True,False
68,"Petrista de corazón que tú inteligencia sea la que tome la decisión, vota inteligente frente al #PetrismoSucio",1535365000117858305,No,Si,False,True
83,Mucho se habla de #FraudeElectoral pero el tema está en no dejarse confundir y estudiar los programas y mirar dónde está la posible solución a la mayoría o algunos problemas sociales. SU CONCIENCIA DECIDE,1532389060072951809,Si,No,True,False
84,#ColombiaNecesita más honestidad y coherencia. Menos egos en los líderes políticos. Y sobre todo más educación que despierte la conciencia.,1532463961643294721,Si,No,True,False


In [60]:
merged_ironia = merged_ironia.fillna('No')
merged_ironia['ironia_juanjo_bool'] = merged_ironia['ironia_juanjo'].map({'Si': True, 'No': False})
merged_ironia['ironia_juanma_bool'] = merged_ironia['ironia_juanma'].map({'Si': True, 'No': False})

In [61]:
arr_num = np.array((merged_ironia['ironia_juanjo_bool'], merged_ironia['ironia_juanma_bool'])).T
agg_num = irr.aggregate_raters(arr_num)
irr.fleiss_kappa(agg_num[0], method='fleiss')

0.4019138755980862

In [64]:
merged_ironia[merged_ironia['ironia_juanjo'] != merged_ironia['ironia_juanma']]

,text,tweet_id,ironia_juanjo,ironia_juanma,emocion_juanjo_bool,emocion_juanma_bool,ironia_juanjo_bool,ironia_juanma_bool
10,#ElDebateDefinitivo fico es aguevado...creia que petro iba a caer en la trampa. Es obvio que tiene que extraditar jajajaj,1528908094675333124,Si,No,True,False,True,False
12,"#ElDebateDefinitivo le falta a Federico Gutiérrez decir que ""vamos a vivir sabroso"" y ya se termina de robar todo el discurso de Petro. Consejo. Si alguien se le roba la idea al otro. No vayan por el que se roba la idea, vayan por el que es el original.",1528921475553669128,No,Si,False,True,False,True
13,"Fajardo tiene la misma propuesta de petro en pensiones, y ni el mismo lo sabe. Petro se lo tuvo que explicar. #ElDebateDefinitivo",1528936500745973760,No,Si,False,True,False,True
14,La Silicon Valley Latinoamericana aún no tiene software para las votaciones. #AColombiaLePreocupa,1529070396087185409,No,Si,False,True,False,True
25,#ElDebateDefinitivo ¿Ya están diciendo que el cafre Petro los peinó a todos?,1528903851981553666,No,Si,False,True,False,True
36,Gracias Fecode esto no se habría hecho sin ustedes que llevan muchos años adoctrinando a los niños para ser de izquierda radical! #PetroEsPresidente,1538846127243534336,Si,No,True,False,True,False
48,"Petro se quedó sin voz de tanta plaza pública, Rodolfo se quedó sin voz de tanto cantar “pepas y agua pa la seca, todo el mundo en pastillas pa la discoteca”. Son datos y hay que darlos. #estedomingo #YoVotoPor #petrosky",1537788369052352512,No,Si,False,True,False,True
66,@chirivica26 @navarrowolff Ahh y q hace a Petro el demócrata del siglo? Todo bien por las bodegas del #PetrismoSucio ? Estan como asustados q no escriben bien,1535442087390371840,No,Si,False,True,False,True
86,"@BluRadioCo Ajá como en argentina, chile y peru. Ni que decir de Venezuela, Nicaragua y cuba también eligieron vivir sabroso 🥶 #RodolfoPresidente2022",1531995875471503360,Si,No,True,False,True,False
89,Espero que todos y todas estemos de acuerdo con que era muchísimo mejor tener en #SegundaVuelta a Fajardo y no a Rodolfo o Fico. Linda noche 😊,1531093498413387776,Si,No,True,False,True,False


In [10]:
agreements = {}
for i in dict_emociones.keys():
    arr = np.array((merged_df[i + '_juanjo'], merged_df[i + '_juanma'])).T
    agg = irr.aggregate_raters(arr)
    agreements[i] = irr.fleiss_kappa(agg[0], method='fleiss')

In [11]:
agreements

{'alegria': 0.7312653562653563,
 'miedo': 0.5699541284403673,
 'sorpresa': -0.012145748987857056,
 'tristeza': 0.5173745173745175,
 'asco': 0.45812807881773404,
 'ira': 0.2941371865138679,
 'otra': -0.016260162601628193}

In [15]:
total = sum(emotions_count.values())
avgd_agreement = 0
mean_agreement = 0
for i in agreements.keys():
    avgd_agreement+= agreements[i]*(emotions_count[i]/total)
    mean_agreement+= agreements[i]
mean_agreement = mean_agreement/len(agreements)

In [16]:
avgd_agreement

0.505665948332678

In [17]:
mean_agreement

0.3632076222603368

In [19]:
pd.options.display.max_colwidth = 300
merged_emocion_disagreement = merged_emocion[(merged_emocion['emocion_juanjo'] == 'No') |(merged_emocion['emocion_juanma'] == 'No')]

In [30]:
to_exclude = merged_relevancia_no[['text','tweet_id']].append(merged_emocion_disagreement[['text','tweet_id']], ignore_index=True).drop_duplicates(subset=['tweet_id'])

In [56]:
disagreement_unique = full_df_disagreement.drop_duplicates(subset=['tweet_id'])[['text','tweet_id']]
disagreement_unique = disagreement_unique.reset_index(drop = True)

In [61]:
df_clean = disagreement_unique[disagreement_unique.merge(to_exclude, how='left', on='tweet_id')['text_y'].isna()]

In [77]:
df_final = df_clean[~df_clean['text'].str.contains('https')]
df_final.to_csv('../Data/Data_Annotated/to_annotate.csv', index=False)